In [1]:
from rouge import Rouge
import glob
import os
import re
import pandas as pd

#goldSummaries = glob.glob(r'..data/raw/OpinosisDataset1.0_0/summaries-gold/*/')
gold_summaries = os.walk(r'../data/raw/OpinosisDataset1.0_0/summaries-gold')
gold_list = next(gold_summaries)[1]

In [2]:
def rouge_evaluation(summaryfiles_dir, summaryDir):
    summaryFiles = glob.glob(summaryfiles_dir)
    rouge_1_scores_list = []
    rouge_2_scores_list = []
    rouge_l_scores_list = []
    for h in summaryFiles:
        foldername_search = re.search(r'[^\\/:*?"<>|\r\n]+$', h)
        foldername = (foldername_search.group()).split('.')[0]
        with open(h, 'r') as f:
            hypothesis = f.read()
            if foldername in gold_list:
                files = glob.glob(summaryDir+foldername+'/*')
                foldername = ' '.join(foldername.split('_'))
                for r in files:
                    with open(r, 'r') as f:
                        reference = f.read()
                        rouge = Rouge()
                        scores = rouge.get_scores(hypothesis, reference)[0]
                        rouge_1_scores_list.append(pd.DataFrame(scores['rouge-1'], index=[foldername]))
                        rouge_2_scores_list.append(pd.DataFrame(scores['rouge-2'], index=[foldername]))
                        rouge_l_scores_list.append(pd.DataFrame(scores['rouge-l'], index=[foldername]))
                        rouge_1_df = pd.concat(rouge_1_scores_list)
                        rouge_2_df = pd.concat(rouge_2_scores_list)
                        rouge_l_df = pd.concat(rouge_l_scores_list)
    return rouge_1_df, rouge_2_df, rouge_l_df

In [3]:
textrank_rouge_1, textrank_rouge_2, textrank_rouge_l = rouge_evaluation(r'../data/processed/textrank/*', '../data/raw/OpinosisDataset1.0_0/summaries-gold/')

In [4]:
lexrank_rouge_1, lexrank_rouge_2, lexrank_rouge_l = rouge_evaluation(r'../data/processed/lexrank/*', '../data/raw/OpinosisDataset1.0_0/summaries-gold/')

In [5]:
opinosis_rouge_1, opinosis_rouge_2, opinosis_rouge_l = rouge_evaluation(r'../data/processed/opinosis/*', '../data/raw/OpinosisDataset1.0_0/summaries-gold/')

In [6]:
textrank_rouge_1.describe().round(3).to_csv('textrank_rouge_1_stats.csv')
textrank_rouge_2.describe().round(3).to_csv('textrank_rouge_2_stats.csv')
textrank_rouge_l.describe().round(3).to_csv('textrank_rouge_l_stats.csv')

In [7]:
lexrank_rouge_1.describe().round(3).to_csv('lexrank_rouge_1_stats.csv')
lexrank_rouge_2.describe().round(3).to_csv('lexrank_rouge_2_stats.csv')
lexrank_rouge_l.describe().round(3).to_csv('lexrank_rouge_l_stats.csv')

In [8]:
opinosis_rouge_1.describe().round(3).to_csv('opinosis_rouge_1_stats.csv')
opinosis_rouge_2.describe().round(3).to_csv('opinosis_rouge_2_stats.csv')
opinosis_rouge_l.describe().round(3).to_csv('opinosis_rouge_l_stats.csv')

In [13]:
#textrank_rouge_1.describe().round(3).to_latex()
#textrank_rouge_2.describe().round(3).to_csv('textrank_rouge_2_stats.csv')
#textrank_rouge_l.describe().round(3).to_csv('textrank_rouge_l_stats.csv')

In [15]:
#lexrank_rouge_1.describe().round(3).to_csv
lexrank_rouge_2.describe().round(3).to_csv
#lexrank_rouge_l.describe().round(3).to_csv

<bound method DataFrame.to_csv of              f        p        r
count  238.000  238.000  238.000
mean     0.037    0.044    0.041
std      0.065    0.084    0.070
min      0.000    0.000    0.000
25%      0.000    0.000    0.000
50%      0.000    0.000    0.000
75%      0.059    0.059    0.069
max      0.378    0.500    0.467>

In [15]:
#opinosis_rouge_1.describe().round(3).to_csv('opinosis_rouge_1_stats.csv')
#opinosis_rouge_2.describe().round(3).to_csv('opinosis_rouge_2_stats.csv')
#opinosis_rouge_l.describe().round(3).to_csv('opinosis_rouge_l_stats.csv')

In [9]:
from scipy.stats import f_oneway

In [10]:
f_oneway(textrank_rouge_1, lexrank_rouge_1, opinosis_rouge_l)

F_onewayResult(statistic=array([ 21.82003794,  99.90448333, 138.21070489]), pvalue=array([6.35507995e-10, 5.79747397e-39, 1.96784205e-51]))

In [11]:
f_oneway(lexrank_rouge_2, lexrank_rouge_2, opinosis_rouge_2)

F_onewayResult(statistic=array([2.38162135, 8.90234002, 0.04781098]), pvalue=array([9.31374301e-02, 1.51837217e-04, 9.53317028e-01]))

In [12]:
f_oneway(lexrank_rouge_l, lexrank_rouge_l, opinosis_rouge_l)

F_onewayResult(statistic=array([ 1.76808725, 21.8449424 ,  0.63825938]), pvalue=array([1.71408624e-01, 6.20770332e-10, 5.28513406e-01]))

In [23]:
survey = pd.read_csv('responses.csv')

In [24]:
survey.head()

,Timestamp,Gender,Age,Is English your first language?,How would you rate your English reading abilities?,readability_human_2,readability_lexrank,readability_textrank,readability_opinosis,readability_human_4,...,utility_lexrank,utility_textrank,utility_opinosis,utility_human_4,utility_human_1,utility_human_3,utility_wordcloud,utility_sentiment,"Looking at the figure above, which combination of summaries would help you in making a decision to buy a Toyota Camry 2007 model?",Please comment on reasons for your selection.
0,4/17/2019 15:05,Male,30-39,Yes,Advanced,2,2,2,4,3,...,4,4,4,4,5,4,3,4,"3, 4",Based on the combination
1,4/17/2019 15:29,Female,30-39,Yes,Advanced,4,5,5,5,5,...,5,5,5,5,5,5,5,5,"1, 3, 6, 8",Length and visual ease
2,4/17/2019 16:39,Male,25-29,Yes,Advanced,5,4,5,4,5,...,4,4,4,5,5,3,5,5,"3, 9",NaN
3,4/17/2019 17:51,Male,30-39,No,Intermediate,3,4,4,5,4,...,4,4,5,5,4,3,5,4,"4, 6, 8, 9",NaN
4,4/17/2019 18:09,Male,25-29,Yes,Advanced,4,4,3,4,4,...,3,3,3,3,3,3,3,3,4,NaN


In [26]:
survey.describe()

,readability_human_2,readability_lexrank,readability_textrank,readability_opinosis,readability_human_4,readability_human_1,readability_human_3,utility_human_2,utility_lexrank,utility_textrank,utility_opinosis,utility_human_4,utility_human_1,utility_human_3,utility_wordcloud,utility_sentiment
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,3.700000,3.666667,4.200000,3.833333,3.900000,4.333333,3.900000,3.366667,2.966667,3.500000,2.800000,3.833333,4.433333,3.366667,3.266667,3.666667
std,0.987857,1.093345,0.924755,1.234094,1.295882,0.922266,0.884736,1.066200,0.964305,0.861034,1.297212,0.746640,0.678911,0.850287,1.201532,1.154701
min,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,3.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,4.000000,4.000000,3.000000,4.000000,3.000000,3.000000,2.000000,3.000000,2.000000,3.000000,4.000000,3.000000,2.000000,3.000000
50%,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,3.000000,4.000000,3.000000,4.000000,5.000000,3.000000,3.000000,4.000000
75%,4.000000,4.000000,5.000000,5.000000,5.000000,5.000000,4.750000,4.000000,4.000000,4.000000,3.750000,4.000000,5.000000,4.000000,4.000000,4.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [29]:
f_oneway(survey.readability_textrank,
        survey.readability_lexrank,
        survey.readability_opinosis,
        survey.readability_human_1,
        survey.readability_human_2,
        survey.readability_human_3,
        survey.readability_human_4)

F_onewayResult(statistic=1.642202729044834, pvalue=0.13714796368667523)

In [31]:
f_oneway(survey.utility_textrank,
        survey.utility_lexrank,
        survey.utility_opinosis,
        survey.utility_human_1,
        survey.utility_human_2,
        survey.utility_human_3,
        survey.utility_human_4,
        survey.utility_wordcloud,
        survey.utility_sentiment)

F_onewayResult(statistic=6.984530492472566, pvalue=2.3987481126500628e-08)

In [36]:
survey[['readability_textrank',
     'readability_lexrank',
       'readability_opinosis',
        'readability_human_1',
        'readability_human_2',
        'readability_human_3',
       'readability_human_4']].describe().to_csv('readability.csv')

In [37]:
survey[['utility_textrank',
     'utility_lexrank',
       'utility_opinosis',
        'utility_human_1',
        'utility_human_2',
        'utility_human_3',
       'utility_human_4',
       'utility_wordcloud',
       'utility_sentiment']].describe().to_csv('utility.csv')

In [10]:
opinosis_rouge_1.idxmax()

f    rooms swissotel chicago
p    rooms swissotel chicago
r    rooms swissotel chicago
dtype: object

In [11]:
opinosis_rouge_2.idxmax()

f           rooms swissotel chicago
p    transmission toyota camry 2007
r           rooms swissotel chicago
dtype: object

In [12]:
opinosis_rouge_l.idxmax()

f    rooms swissotel chicago
p    rooms swissotel chicago
r    rooms swissotel chicago
dtype: object

In [23]:
opinosis_rouge_1.loc["location holiday inn london"].max()

f    0.346154
p    0.281250
r    0.500000
dtype: float64

In [17]:
opinosis_rouge_2.loc["location holiday inn london"].mean()

f    0.021895
p    0.014706
r    0.047727
dtype: float64

In [19]:
opinosis_rouge_l.loc["location holiday inn london"].mean()

f    0.135737
p    0.125000
r    0.345076
dtype: float64

In [20]:
textrank_rouge_1.idxmax()

f    service bestwestern hotel sfo
p    service bestwestern hotel sfo
r              sound ipod nano 8gb
dtype: object

In [27]:
opinosis_rouge_l.describe().round(3)
#opinosis_rouge_2.describe().round(3)
#opinosis_rouge_l.describe().round(3)

,f,p,r
count,238.000,238.000,238.000
mean,0.184,0.303,0.193
std,0.118,0.212,0.125
min,0.000,0.000,0.000
25%,0.101,0.143,0.115
50%,0.167,0.279,0.175
75%,0.236,0.400,0.248
max,0.791,1.000,0.714


In [49]:
opinosis_rouge_1.sort_values('r')

,f,p,r
satellite garmin nuvi 255W gps,0.000000,0.000000,0.000000
accuracy garmin nuvi 255W gps,0.000000,0.000000,0.000000
food swissotel chicago,0.000000,0.000000,0.000000
interior toyota camry 2007,0.000000,0.000000,0.000000
navigation amazon kindle,0.000000,0.000000,0.000000
parking bestwestern hotel sfo,0.000000,0.000000,0.000000
display garmin nuvi 255W gps,0.000000,0.000000,0.000000
display garmin nuvi 255W gps,0.000000,0.000000,0.000000
rooms swissotel chicago,0.000000,0.000000,0.000000
satellite garmin nuvi 255W gps,0.000000,0.000000,0.000000


In [46]:
opinosis_rouge_1.loc['staff bestwestern hotel sfo']

,f,p,r
staff bestwestern hotel sfo,0.363636,0.285714,0.500000
staff bestwestern hotel sfo,0.173913,0.285714,0.125000
staff bestwestern hotel sfo,0.235294,0.285714,0.200000
staff bestwestern hotel sfo,0.173913,0.285714,0.125000
staff bestwestern hotel sfo,0.222222,0.285714,0.181818
